In [23]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import http.cookiejar as cookielib
import time
from bs4 import BeautifulSoup
import re

In [5]:
url="http://kissanime.ru"

In [6]:
driver=webdriver.Chrome()
driver.get(url)
time.sleep(10)
cs=driver.get_cookies()
driver.close()

In [7]:
header={"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36"}
cj=cookielib.CookieJar()
for c in cs:
    if 'expiry' not in c.keys():continue
    ck=cookielib.Cookie(name=c['name'], value=c['value'], domain=c['domain'], path=c['path'],secure=c['secure'], rest=False, version=0,port=None,port_specified=False,domain_specified=False,domain_initial_dot=False,path_specified=True,expires=c['expiry'],discard=True,comment=None,comment_url=None,rfc2109=False)
    cj.set_cookie(ck)

In [8]:
s=requests.Session()
r=s.get(url+"/Login",cookies=cj,headers=header)

In [9]:
print(r.status_code)
soup=BeautifulSoup(r.content,"html.parser")


200


In [10]:
r=s.post("http://kissanime.ru/Login",data={'username':'srn2012','password':'soumya@123'},cookies=cj,headers=header)
r.status_code

200

In [11]:
r=s.post("http://kissanime.ru/Search/SearchSuggestx",data={'type':'Anime','keyword':'dragon ball'},cookies=cj,headers=header)
r.status_code

200

In [12]:
soup=BeautifulSoup(r.content,"html.parser")
results=soup.find_all("a")

In [13]:
animelink=results[0]["href"]

In [14]:
r=s.get(animelink,cookies=cj,headers=header)
r.status_code

200

In [15]:
soup=BeautifulSoup(r.content,"html.parser")
episodeTable=soup.find("table",{"class":"listing"}).find_all("td")
links=[]
for c,x in enumerate(episodeTable):
    if(c%2==0): links.append(x.find("a")["href"])

In [26]:
r=s.get("http://kissanime.ru"+links[1]+"&s=rapidvideo",cookies=cj,headers=header)
r.status_code
print("http://kissanime.ru"+links[1]+"&s=rapidvideo")

http://kissanime.ru/Anime/Dragon-Ball-Dub/Episode-152?id=51476&s=rapidvideo


In [28]:
r.content
soup=BeautifulSoup(r.content,"html.parser")
#soup.find("iframe",{"id":"my_video_1"})["src"]
soup.prettify()

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml">\n <head id="ctl00_Head1">\n  <title>\n   Dragon Ball (Dub)\r\n    Episode 152\r\n    - Watch\r\n    Dragon Ball (Dub)\r\n    Episode 152\r\n    online in high quality\n  </title>\n  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>\n  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n  <meta content="100006233868825" property="fb:admins"/>\n  <meta content="130397647116314" property="fb:app_id"/>\n  <script src="/Scripts/jquery17.min.js" type="text/javascript">\n  </script>\n  <!-- CSS -->\n  <link href="/Content/css/tpl_style.css?v=7" rel="stylesheet" type="text/css"/>\n  <link href="/Content/css/upload-progress.css" rel="stylesheet" type="text/css"/>\n  <link href="/Content/images/favicon.ico" rel="shortcut icon" type="image/x-icon">\n   <script src="/Scripts/google.js" type="text/javascript">\

In [38]:
rapidurl= "https://www.rapidvideo.com/"+re.search(r'https://www.rapidvideo.com/(.*)(?=" allowfullscreen)',soup.prettify()).group(1)
rapidurl

'https://www.rapidvideo.com/e/FF1FZY5XSI'

In [39]:
r=s.get(rapidurl)
r.status_code

200

In [40]:
soup=BeautifulSoup(r.content,"html.parser")
soup.prettify()

'<!DOCTYPE html>\n<html>\n <head>\n  <title>\n   DBNLDragonBall152MysteryoftheDarkWorldDRx264C226B0F4enr229.1r443-rh-860.mp4\n  </title>\n  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>\n  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>\n  <link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>\n  <link href="/favicon.ico" rel="icon" type="image/x-icon"/>\n  <link href="/css/jquery-ui.css" rel="stylesheet" type="text/css"/>\n  <link href="/css/opensans.css" rel="stylesheet" type="text/css"/>\n  <link href="/css/layout.css?v=1.5.0" rel="stylesheet" type="text/css"/>\n  <script src="/js/jquery.min.js" type="text/javascript">\n  </script>\n  <script src="/js/jquery.validate.min.js" type="text/javascript">\n  </script>\n  <script src="/js/jquery-ui.min.js" type="text/javascript">\n  </script>\n  <script src="/js/jquery.limit-1.2.source.js" type="text/javascript">\n  </script>\n  <script src="/js/jquery.tipsy.js" type="text/javascri

In [49]:
dllink=soup.find("video",{"class":"video-js vjs-big-play-centered"}).find("source")["src"]

In [ ]:
with open('a.mp4','wb') as file:
    r = requests.get(dllink,stream=True)
    file.write(r.content)